In [1]:
!pip install transformers
!pip install sentencepiece
#30s

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.2 MB/s eta 0:00:00


In [14]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import keras
import transformers
import sentencepiece
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import sentencepiece as spm
from transformers import T5Tokenizer
#10s

In [3]:
df = pd.read_csv("/content/tcc_ceds_music.csv")

In [5]:
import sentencepiece as spm
from transformers import T5Tokenizer
# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

c:\Users\lenab\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\lenab\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lenab\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see ac

In [5]:
# Convert all entries to string and replace NaN values with an empty string
df['lyrics'] = df['lyrics'].fillna("").astype(str)

In [6]:
# Extract the names of the genre columns
genre_columns = df.columns[7:29].tolist()
# Create the genres dictionary
genres = {i: genre_columns[i] for i in range(len(genre_columns))}
print(genres)

{0: 'dating', 1: 'violence', 2: 'world/life', 3: 'night/time', 4: 'shake the audience', 5: 'family/gospel', 6: 'romantic', 7: 'communication', 8: 'obscene', 9: 'music', 10: 'movement/places', 11: 'light/visual perceptions', 12: 'family/spiritual', 13: 'like/girls', 14: 'sadness', 15: 'feelings', 16: 'danceability', 17: 'loudness', 18: 'acousticness', 19: 'instrumentalness', 20: 'valence', 21: 'energy'}


In [7]:
df['tokenized_lyrics'] = df['lyrics'].apply(lambda x: tokenizer.encode(x, truncation=True, max_length=512)) #18s

In [8]:
# Select the relevant columns using the genres dictionary
selected_columns = [genres[i] for i in range(len(genres))]

# Fuse the columns into a single column containing lists
df['fused_genre_values'] = df[selected_columns].apply(lambda row: row.tolist(), axis=1)

In [9]:
df['fused_genre_values'] = df[selected_columns].apply(lambda row: np.array(row), axis=1)

In [10]:
# Drop the selected columns
df = df.drop(columns=selected_columns)

In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
train_data

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,topic,age,tokenized_lyrics,fused_genre_values
12548,39887,ella fitzgerald,from this moment on,1956,blues,close nights morose beguine begin future look ...,35,romantic,0.914286,"[885, 8348, 2288, 8115, 36, 17996, 15, 1731, 6...","[0.0719181524370081, 0.0017543861600171, 0.089..."
8688,25035,elvin bishop,struttin' my stuff,1975,country,struttin stuff struttin stuff struttin stuff s...,66,obscene,0.642857,"[3, 6159, 17, 17, 77, 2005, 3, 6159, 17, 17, 7...","[0.0012836970522399, 0.0012836970569885, 0.001..."
17255,54658,dean martin,don't you remember?,1954,jazz,remember remember remember remember moon look ...,74,world/life,0.942857,"[1423, 1423, 1423, 1423, 8114, 320, 706, 1423,...","[0.0007974481775604, 0.0007974482075983, 0.625..."
3737,10960,matt monro,from russia with love,1990,pop,russia wiser goodbye travel world learn return...,25,world/life,0.428571,"[3, 26165, 7624, 52, 23281, 1111, 296, 669, 12...","[0.0027700831391586, 0.1102372266707705, 0.572..."
19343,59628,boney james,it's all good,1997,jazz,good good good good good good good good good g...,11,feelings,0.328571,"[207, 207, 207, 207, 207, 207, 207, 207, 207, ...","[0.0043859649122807, 0.0043859649122807, 0.004..."
...,...,...,...,...,...,...,...,...,...,...,...
21575,64844,ludwig von 88,mike tyson,1988,reggae,greatest punchin right punchin leave punchin f...,74,violence,0.457143,"[4016, 11462, 77, 269, 11462, 77, 1175, 11462,...","[0.0013157894991302, 0.3780922677130172, 0.001..."
5390,15525,o.a.r.,love and memories,2005,pop,lovely lovely vision stick inside memory forge...,83,violence,0.214286,"[3061, 3061, 2267, 4372, 1096, 2594, 2612, 256...","[0.0007209805373356, 0.4993907162778752, 0.000..."
860,2282,? & the mysterians,96 tears,1964,pop,teardrops heart cryin teardrops heart leave la...,51,sadness,0.800000,"[11482, 15946, 7, 842, 12676, 77, 11482, 15946...","[0.0675775586745745, 0.0012531328608186, 0.001..."
15795,50496,the coasters,run red run,2005,blues,go buy monkey pawn shop broker teach monkey gu...,92,obscene,0.214286,"[281, 805, 21622, 3, 19589, 29, 1814, 11630, 3...","[0.0009930486758507, 0.3109565000428251, 0.000..."


In [13]:
vocab_size = len(tokenizer.get_vocab())  # Size of your tokenizer's vocabulary
embedding_dim = 128  # Or any other desired size

In [14]:
model = keras.Sequential([
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=98),
    keras.layers.Flatten(),

    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(22, activation='softmax')
])
#it has to be a tuple for the first one

In [15]:
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Convert tokenized_lyrics to a NumPy array and pad sequences
X_train = pad_sequences(train_data['tokenized_lyrics'], maxlen=98)

# If fused_genre_values is categorical labels, one-hot encode them
# Assuming you have 21 classes as indicated by the output layer of your model
Y_train = np.stack(train_data['fused_genre_values'].values)

# Fit the model
model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
710/710 [==============================] - 39s 44ms/step - loss: 0.2974 - accuracy: 0.2593
Epoch 2/10
710/710 [==============================] - 9s 13ms/step - loss: 0.2666 - accuracy: 0.4348
Epoch 3/10
710/710 [==============================] - 6s 8ms/step - loss: 0.2507 - accuracy: 0.6067
Epoch 4/10
710/710 [==============================] - 5s 8ms/step - loss: 0.2435 - accuracy: 0.6931
Epoch 5/10
710/710 [==============================] - 5s 7ms/step - loss: 0.2405 - accuracy: 0.7319
Epoch 6/10
710/710 [==============================] - 4s 6ms/step - loss: 0.2381 - accuracy: 0.7582
Epoch 7/10
710/710 [==============================] - 4s 6ms/step - loss: 0.2364 - accuracy: 0.7790
Epoch 8/10
710/710 [==============================] - 4s 6ms/step - loss: 0.2351 - accuracy: 0.7900
Epoch 9/10
710/710 [==============================] - 4s 5ms/step - loss: 0.2342 - accuracy: 0.8054
Epoch 10/10
710/710 [==============================] - 4s 6ms/step - loss: 0.2333 - accuracy: 0.8

In [17]:
X_test = pad_sequences(test_data['tokenized_lyrics'], maxlen=98)
Y_test = np.stack(test_data['fused_genre_values'].values)

In [18]:
test_loss , test_acc = model.evaluate(X_test , Y_test ,
verbose =2)

178/178 - 1s - loss: 0.2946 - accuracy: 0.2895 - 751ms/epoch - 4ms/step


In [19]:
# 1. Tokenize the text
sample_text = "good good good good good good good good good"
tokenized_sample = [tokenizer.encode(sample_text, truncation=True, max_length=512)]

In [20]:
# 2. Pad the sequence
padded_sample = pad_sequences(tokenized_sample, maxlen=98)

In [21]:
# 3. Make a prediction
prediction = model.predict(padded_sample)

1/1 [==============================] - 0s 128ms/step


In [23]:
# 4. Interpret the prediction
predicted_class = np.argmax(prediction)
predicted_genre = genres[predicted_class]
print(f"The predicted genre is: {predicted_genre}")

The predicted genre is: feelings


In [24]:
model_path = "my_model_datacamp.h5"
model.save(model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
from tensorflow.keras.models import load_model

model_path = "my_model_datacamp.h5"
loaded_model = load_model(model_path)

In [23]:
genres = {0: 'dating', 1: 'violence', 2: 'world/life', 3: 'night/time', 4: 'shake the audience', 5: 'family/gospel', 6: 'romantic', 7: 'communication', 8: 'obscene', 9: 'music', 10: 'movement/places', 11: 'light/visual perceptions', 12: 'family/spiritual', 13: 'like/girls', 14: 'sadness', 15: 'feelings', 16: 'danceability', 17: 'loudness', 18: 'acousticness', 19: 'instrumentalness', 20: 'valence', 21: 'energy'}

#Voilà comment utiliser le model que tu as load

In [140]:
# Tokenize the text
sample_text = "Tawol Athodu Ek Erilaz Owlthuthewaz Niwaremariz Saawilagar Hateka Harja Fehu, Uruz, Thurisaz, Ansuz, Raidho, Kenaz, Gebo, Wunjo, Hagal, Naudhiz, Isa, Jera, Eihwaz, Perthro, Algiz, Sowelo, Tiwaz, Berkano, Ehwaz, Mannaz, Laguz, Ingwaz, Dagaz, Othala. Wotani Ruoperath Gleiaugiz Eiurzi Ai au is urki Uiniz Ik"
#sample_text = "Roar! Tipe, tipe, zangalewa (World Cup! World Cup!) Tipe, tipe, zangalewa (World Cup! World Cup!) Tipe, tipe, zangalewa (World Cup! World Cup!) Tipe, tipe, zangalewa (World Cup! World Cup!) You're a good soldier, choosing your battles Pick yourself up and dust yourself off, get back in the saddle You're on the frontline, everyone's watching You know it's serious, we're getting closer, this isn't over The pressure's on, you feel it But you got it all, believe it When you fall get up, oh, oh And if you fall get up, eh, eh Tsamina mina zangalewa 'cause this is Africa Tsamina mina, eh, eh Waka waka, eh, eh Tsamina mina zangalewa This time for Africa Listen to your God, this is our motto Your time to shine, don't wait in line, y vamos por todo People are raising their expectations Go on and feed them, this is your moment, no hesitation Today's your day, I feel it You paved the way, believe it If you get down get up, oh, oh When you get down get up, eh, eh Tsamina mina zangalewa This time for Africa Tsamina mina, eh, eh Waka waka, eh, eh Tsamina mina zangalewa, anawa-a-a Tsamina mina, eh, eh Waka waka, eh, eh Tsamina mina zangalewa This time for Africa Abuyile amajoni pikipiki mama, one A up to Z Bathi susa lamajoni pikipiki mama from East to West Bathi waka waka mah eh, eh Waka waka mah eh, eh Zonk' iZizwe masibuye 'cause this is Africa (Africa, Africa...) Tsamina mina, anawa-a-a Tsamina mina Tsamina mina, anawa-a-a Tsamina mina, eh, eh Waka waka, eh, eh Tsamina mina zangalewa anawa-a-a Tsamina mina, eh, eh Waka waka, eh, eh Tsamina mina zangalewa This time for Africa Tango, eh, eh Tango, eh, eh Tsamina mina zangalewa anawa-a-a Tango, eh, eh Tango, eh, eh Tsamina mina zangalewa, anawa-a-a This time for Africa This time for Africa We all Africa (zama qhela) We all Africa (kwela kwela) Zama qhela"
#sample_text = "good good good good good good good good good"
tokenized_sample = [tokenizer.encode(sample_text, truncation=True, max_length=512)]

# Pad the sequence
padded_sample = pad_sequences(tokenized_sample, maxlen=98)

# Make a prediction
prediction = loaded_model.predict(padded_sample)

# Interpret the prediction
predicted_class = np.argmax(prediction)
predicted_genre = genres[predicted_class]
print(f"The predicted genre is: {predicted_genre}")

1/1 [==============================] - 0s 124ms/step
The predicted genre is: acousticness


In [128]:
de = pd.read_csv("tcc_ceds_music.csv")

In [120]:
print(de.head())

   Unnamed: 0           artist_name            track_name  release_date genre  \
0           0                mukesh  mohabbat bhi jhoothi          1950   pop   
1           4         frankie laine             i believe          1950   pop   
2           6           johnnie ray                   cry          1950   pop   
3          10           pérez prado              patricia          1950   pop   
4          12  giorgos papadopoulos    apopse eida oneiro          1950   pop   

                                              lyrics  len    dating  violence  \
0  hold time feel break feel untrue convince spea...   95  0.000598  0.063746   
1  believe drop rain fall grow believe darkest ni...   51  0.035537  0.096777   
2  sweetheart send letter goodbye secret feel bet...   24  0.002770  0.002770   
3  kiss lips want stroll charm mambo chacha merin...   54  0.048249  0.001548   
4  till darling till matter know till dream live ...   48  0.001350  0.001350   

   world/life  ...   sadne

In [121]:
print(de['topic'])

0           sadness
1        world/life
2             music
3          romantic
4          romantic
            ...    
28367       obscene
28368       obscene
28369       obscene
28370       obscene
28371       obscene
Name: topic, Length: 28372, dtype: object


In [125]:
de['topic'] == 'loudness'

0        False
1        False
2        False
3        False
4        False
         ...  
28367    False
28368    False
28369    False
28370    False
28371    False
Name: topic, Length: 28372, dtype: bool

In [130]:
de[de['topic'] == 'romantic']

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
8,20,stélios kazantzídis,klapse me mana klapse me,1950,pop,lonely night surround power read mind hour nig...,30,0.083537,0.002024,0.002024,...,0.002024,0.031600,0.496372,0.666180,0.974900,0.000014,0.621805,0.400382,romantic,1.000000
14,36,les paul,the carioca,1951,pop,see carioca foxtrot polka little rhythm blue r...,60,0.001253,0.001253,0.081360,...,0.001253,0.049285,0.550525,0.682409,0.495983,0.504049,0.849547,0.642631,romantic,0.985714
22,58,ghantasala,ee jagamantha natika,1952,pop,know baby hang touch weak strong know cope dow...,41,0.038509,0.001645,0.001645,...,0.001645,0.049252,0.440052,0.606005,0.978916,0.000776,0.794930,0.399381,romantic,0.971429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27211,80207,sufjan stevens,visions of gideon,2017,rock,love time video video touch time video video l...,75,0.002506,0.002506,0.002506,...,0.002506,0.002506,0.206109,0.447658,0.945783,0.684211,0.181781,0.314293,romantic,0.042857
27454,80926,x ambassadors,hold you down,2019,rock,ones dance light hold hold ones dance faceless...,105,0.000650,0.000650,0.000650,...,0.000650,0.000650,0.558107,0.771838,0.049296,0.000001,0.435284,0.627616,romantic,0.014286
27800,81550,night shield,"monsters (cruelty 5) (feat. frank waln, young ...",2012,hip hop,nights long go away think true miss voice touc...,22,0.002288,0.002288,0.002288,...,0.041195,0.192486,0.671829,0.782401,0.115461,0.000000,0.770198,0.776770,romantic,0.114286
27982,81853,t-rock,4:20/reincarnated,2016,hip hop,feel feel felt spell feel like time fake finge...,167,0.010711,0.198028,0.000424,...,0.000424,0.060376,0.795299,0.621593,0.000072,0.000000,0.207543,0.454437,romantic,0.057143


In [115]:
print(predicted_genre)

feelings


In [114]:
print(de.head(5))

   Unnamed: 0           artist_name            track_name  release_date genre  \
0           0                mukesh  mohabbat bhi jhoothi          1950   pop   
1           4         frankie laine             i believe          1950   pop   
2           6           johnnie ray                   cry          1950   pop   
3          10           pérez prado              patricia          1950   pop   
4          12  giorgos papadopoulos    apopse eida oneiro          1950   pop   

                                              lyrics  len    dating  violence  \
0  hold time feel break feel untrue convince spea...   95  0.000598  0.063746   
1  believe drop rain fall grow believe darkest ni...   51  0.035537  0.096777   
2  sweetheart send letter goodbye secret feel bet...   24  0.002770  0.002770   
3  kiss lips want stroll charm mambo chacha merin...   54  0.048249  0.001548   
4  till darling till matter know till dream live ...   48  0.001350  0.001350   

   world/life  ...   sadne

In [106]:
grouped_de = de.groupby(predicted_genre)
top_tracks = grouped_de[['artist_name', 'track_name']]
print(top_tracks.head(5))

                artist_name                     track_name
0                    mukesh           mohabbat bhi jhoothi
1             frankie laine                      i believe
2               johnnie ray                            cry
3               pérez prado                       patricia
4      giorgos papadopoulos             apopse eida oneiro
...                     ...                            ...
28367               mack 10                10 million ways
28368                m.o.p.  ante up (robbin hoodz theory)
28369                  nine                  whutcha want?
28370            will smith                         switch
28371                 jeezy                         r.i.p.

[28301 rows x 2 columns]


In [107]:
top_tracks.dtypes

,artist_name,track_name
feelings,,
0.000289,object,object
0.000292,object,object
0.000297,object,object
0.000299,object,object
0.000304,object,object
...,...,...
0.806388,object,object
0.814081,object,object
0.820967,object,object


In [99]:
grouped_de = de.groupby(predicted_genre)

In [100]:
group = de.groupby(de['genre'])

In [101]:
print(group['genre'].head())

0            pop
1            pop
2            pop
3            pop
4            pop
7042     country
7043     country
7044     country
7045     country
7046     country
12487      blues
12488      blues
12489      blues
12490      blues
12491      blues
17091       jazz
17092       jazz
17093       jazz
17094       jazz
17095       jazz
20936     reggae
20937     reggae
20938     reggae
20939     reggae
20940     reggae
23434       rock
23435       rock
23436       rock
23437       rock
23438       rock
27468    hip hop
27469    hip hop
27470    hip hop
27471    hip hop
27472    hip hop
Name: genre, dtype: object


In [139]:
reco = group['artist_name']